<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/SVD(SingularVectorDecomposition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Surprise 패키지를 활용한 SVD 모델 구현

## SVD 모델의 원리

Singular Vector Decomposition을 이용한 모델.

singular matrix = 역행렬이 없는 매트릭스를 의미
det(A) = 0

```
a = [[a,b],
     [c,d]]
일때,
det(A) = ad - bc

A^-1 = det{A}^-1
```

A: 기존 Input data matrix [m x n] 행렬

U: 왼쪽 singular matrix -> y에 따라 x가 무한이거나 존재하지 않음

$\sum$: r개의 특이값을 갖는 대각행렬

V: 오른쪽 singluar matrix

특이값(Singular Value): A에 대해 $A^TA$한 대칭행렬의 고유값을 구해 루트를 씌워준 값을 의미

고유값(Eigen Value): 임의의 벡터 v에 선형변환 A를 적용했을 때, Av = $\lambda$v를 만족하는 상황에서의 $\lambda$값

고유벡터(Eigen Vector): 위 상황에서의 v값을 의미하며, 고유값을 구한 뒤, 가우스 소거법을 이용해 계산한다. 고유벡터는 영벡터가 아닌 벡터를 의미

SVD 특이값 분해의 이용: 정방행렬이 아닌 m x n 의 행렬 A에 대해서도 대칭행렬을 만들 수 있고 이를 바탕으로 행렬 분해가 가능하며, 차원의 축소 역시도 가능하게 만든다.







In [ ]:
print(data)

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data[:10], measures=['rmse', 'mae'], cv = 5, verbose = True)

## Numpy package의 SVD를 이용해보기

In [14]:
import numpy as np
from numpy.linalg import svd

# 4 x 4 행렬 생성
np.random.seed(42)
a = np.random.randn(4,4)  # 평균 0, 표준편차 1의 가우시간 난수 생성
print(np.round(a, 3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


### $U$, $\sum$, $V^T$ 확인해보기

In [15]:
# SVD 함수 적용해서 각 U, Sigma, Vt 확인해보기
U, Sigma, Vt = svd(a)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3))  # 시그마의 경우, 대각행렬의 요소값인 Singular Value를 내림차순으로 가져온다.
print('Vt:', np.round(Vt,3))

U: [[-0.373 -0.598  0.642 -0.302]
 [-0.488 -0.35  -0.745 -0.289]
 [ 0.113  0.444  0.062 -0.887]
 [ 0.781 -0.568 -0.168 -0.197]]
Sigma: [3.08  1.926 0.92  0.342]
Vt: [[ 0.021 -0.412 -0.783 -0.466]
 [-0.291  0.775 -0.086 -0.554]
 [ 0.461  0.479 -0.544  0.512]
 [ 0.838  0.017  0.289 -0.462]]


In [16]:
# 시그마의 대각행렬을 원상복구 해주는 방법: np.diag
Sigma_matrix = np.diag(Sigma)
print('Sigma :', np.round(Sigma_matrix, 3))

Sigma : [[3.08  0.    0.    0.   ]
 [0.    1.926 0.    0.   ]
 [0.    0.    0.92  0.   ]
 [0.    0.    0.    0.342]]


### A = $U \sum V^T$ 만들어보기

In [19]:
a_ = np.dot(np.dot(U,Sigma_matrix), Vt)
print(np.round(a, 3) )  # 맨 처음의 매트릭스 a와 같음을 확인할 수 있다.

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [-0.469  0.543 -0.463 -0.466]
 [ 0.242 -1.913 -1.725 -0.562]]


### COMPACT SVD 구현해보기


In [20]:
# 데이터 의존도가 서로 존재하는 행렬을 만들어본다.
a[2] = a[0]+a[1]
a[3] = a[0]

In [21]:
print(np.round(a,3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [ 0.263 -0.372  2.227  2.29 ]
 [ 0.497 -0.138  0.648  1.523]]


In [22]:
# SVD 분할진행
U, Sigma, Vt = svd(a)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3))  # 시그마의 3,4번째 특잇값이 0임을 확인할 수 있다.
print('Vt:', np.round(Vt,3))

U: [[-0.381  0.505  0.775  0.003]
 [-0.374 -0.679  0.256  0.579]
 [-0.755 -0.174 -0.256 -0.579]
 [-0.381  0.505 -0.519  0.575]]
Sigma: [4.266 1.19  0.    0.   ]
Vt: [[-0.115  0.111 -0.648 -0.745]
 [ 0.517  0.071 -0.677  0.52 ]
 [-0.848 -0.    -0.328  0.416]
 [ 0.024 -0.991 -0.121 -0.047]]


In [28]:
# 특이값 행렬 시그마의 0을 제거한 Compact SVD를 진행한다.
U_ = U[:, :2]
Sigma_ = np.diag(Sigma[:2])
Vt_ = Vt[:2]
print('U_:', np.round(U_,3))
print('Sigma_:', np.round(Sigma_,3))  
print('Vt_:', np.round(Vt_,3))

U_: [[-0.381  0.505]
 [-0.374 -0.679]
 [-0.755 -0.174]
 [-0.381  0.505]]
Sigma_: [[4.266 0.   ]
 [0.    1.19 ]]
Vt_: [[-0.115  0.111 -0.648 -0.745]
 [ 0.517  0.071 -0.677  0.52 ]]


In [29]:
# 다시 복구하기

a_ = np.dot(np.dot(U_, Sigma_), Vt_)
print(np.round(a_, 3))

[[ 0.497 -0.138  0.648  1.523]
 [-0.234 -0.234  1.579  0.767]
 [ 0.263 -0.372  2.227  2.29 ]
 [ 0.497 -0.138  0.648  1.523]]


## Scipy 패키지를 이용해 SVD 구현하기

full_matrices = True -> U를 (M,M), Vt를 (N,N)으로 분할
full_matrices = False -> U를 (M,k), Vt를 (k,N)으로 분할



In [31]:
from scipy.sparse.linalg import svds
from scipy.linalg import svd

# 원본 행렬 생성
np.random.seed(12)
matrix = np.random.random((6,6))
print( np.round(matrix, 3))

[[0.154 0.74  0.263 0.534 0.015 0.919]
 [0.901 0.033 0.957 0.137 0.284 0.606]
 [0.944 0.853 0.002 0.521 0.552 0.485]
 [0.768 0.161 0.765 0.021 0.135 0.116]
 [0.31  0.671 0.471 0.816 0.29  0.733]
 [0.703 0.328 0.335 0.978 0.625 0.95 ]]


In [32]:
U, Sigma, Vt = svd(matrix, full_matrices = False)
print('U:', np.round(U,3))
print('Sigma:', np.round(Sigma,3)) 
print('Vt:', np.round(Vt,3))

U: [[-0.363  0.377  0.435  0.501 -0.449  0.292]
 [-0.385 -0.64   0.235 -0.061 -0.383 -0.487]
 [-0.443  0.149 -0.818  0.259 -0.13  -0.17 ]
 [-0.25  -0.582 -0.058  0.308  0.386  0.593]
 [-0.432  0.254  0.29   0.107  0.695 -0.413]
 [-0.524  0.151  0.006 -0.756 -0.055  0.357]]
Sigma: [3.145 1.289 0.799 0.603 0.301 0.058]
Vt: [[-0.482 -0.369 -0.329 -0.429 -0.269 -0.517]
 [-0.497  0.396 -0.611  0.414 -0.004  0.227]
 [-0.555 -0.226  0.54   0.099 -0.373  0.446]
 [ 0.008  0.768  0.177 -0.418 -0.442 -0.092]
 [-0.212  0.181  0.397  0.535  0.106 -0.684]
 [ 0.41  -0.182 -0.193  0.417 -0.762 -0.079]]


### Scipy svd를 통해 Truncated SVD 구현해보기

In [33]:
#사용할 특이값 개수 지정
num_comp = 4

U_, Sigma_, Vt_ = svds(matrix, k = num_comp)
print('U_:', np.round(U_,3))          # 4개의 특이값에 맞춰 행렬곱을 취할 수 있도록 행렬이 분할되었다.
print('Sigma_:', np.round(Sigma_,3))  # 시그마 행렬에서는 4개의 특이값만을 가진 대각행렬이 된 것을 볼 수 있다.
print('Vt_:', np.round(Vt_,3))        # U와 마찬가지.

U_: [[ 0.501  0.435  0.377  0.363]
 [-0.061  0.235 -0.64   0.385]
 [ 0.259 -0.818  0.149  0.443]
 [ 0.308 -0.058 -0.582  0.25 ]
 [ 0.107  0.29   0.254  0.432]
 [-0.756  0.006  0.151  0.524]]
Sigma_: [0.603 0.799 1.289 3.145]
Vt_: [[ 0.008  0.768  0.177 -0.418 -0.442 -0.092]
 [-0.555 -0.226  0.54   0.099 -0.373  0.446]
 [-0.497  0.396 -0.611  0.414 -0.004  0.227]
 [ 0.482  0.369  0.329  0.429  0.269  0.517]]


In [35]:
# 행렬 다시 합치기 -> Truncated 이므로 0이 아닌 Singular Value도 없는 상태에서의 행렬곱을 취한것이므로, 그 값이 완전히 맨 처음의 값과 일치하진 않는다.

a_ = np.dot(np.dot(U_, np.diag(Sigma_)), Vt_)
print(np.round(a_, 3))

[[ 0.119  0.768  0.32   0.599  0.042  0.828]
 [ 0.888  0.049  0.997  0.211  0.274  0.525]
 [ 0.94   0.858  0.016  0.546  0.549  0.458]
 [ 0.779  0.146  0.725 -0.056  0.149  0.198]
 [ 0.364  0.629  0.383  0.714  0.249  0.874]
 [ 0.691  0.334  0.345  0.978  0.642  0.941]]


## Scikit-Learn패키지를 이용해 SVD 해보기

In [40]:
# 원본 행렬 생성
np.random.seed(12)
matrix = np.random.random((6,6))
print( np.round(matrix, 3))

[[0.154 0.74  0.263 0.534 0.015 0.919]
 [0.901 0.033 0.957 0.137 0.284 0.606]
 [0.944 0.853 0.002 0.521 0.552 0.485]
 [0.768 0.161 0.765 0.021 0.135 0.116]
 [0.31  0.671 0.471 0.816 0.29  0.733]
 [0.703 0.328 0.335 0.978 0.625 0.95 ]]


In [43]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=4)
tsvd.fit(matrix)
a_tr = tsvd.transform(matrix)
a_tr

array([[ 1.14188038, -0.48539404, -0.34749425, -0.30191158],
       [ 1.20960458,  0.82529355, -0.1876061 ,  0.03686302],
       [ 1.39327376, -0.19179119,  0.65318726, -0.15636906],
       [ 0.78638259,  0.75033534,  0.04616277, -0.18548138],
       [ 1.35902108, -0.32771741, -0.23137577, -0.06471603],
       [ 1.64798258, -0.1950719 , -0.00497557,  0.4562135 ]])

## Surprise 패키지를 이용해 SVD 구현하기

### 인접행렬 및 유사도를 통한 가장 비슷한 유저 찾기

In [85]:
!pip install scikit-surprise

In [86]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [87]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)

In [88]:
# user와 movie의 index를 0부터 시작할 수 있도록 1씩 빼준다.
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
# shape 자체는 위에서 1씩 빼준것을 감안하여 1씩 더해준 것을 shape으로 둔다.
shape= (n_users +1, n_movies +1)
shape

(943, 1682)

In [89]:
# 인접행렬 생성
adj_matrix = np.ndarray(shape, dtype=int)
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [90]:
adj_matrix.shape  # 943개의 샘플(유저)과 1682개의 features(영화)에 대한 데이터

(943, 1682)

In [91]:
for  user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.  # 데이터가 있는 경우, 1로 설정해준다.
adj_matrix


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [103]:
len(adj_matrix)

943

In [108]:
len(adj_matrix[0])

1682

In [109]:
# 추천 진행할 id와 matrix에서의 해당 id 설정
id, vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  # 유저 id와 선택 id가 다른경우, 유사도를 체크 한다. 선택 id에 대해서는 할 필요가 없기 때문에 생랼
  if id != user_id:
    similarity = np.dot(vector, user_vector)
    if similarity > best_match:  # 유사도가 기존 best_match보다 높을 경우
      best_match = similarity # best_match에 해당 유사도 값을 넣어준다.
      best_match_id = user_id # 현재 유사도를 계산한 유저 id를 best match id로 선정
      best_match_vector = user_vector # 현재 유사도를 계산한 vector를 넣는다.

print(f'Best Match (유사도 스코어): {best_match},\nBest Match ID (id "{id}"번과 가장 유사한 id): {best_match_id}')


Best Match (유사도 스코어): 183,
Best Match ID (id "0"번과 가장 유사한 id): 275


### 특정 ID('0')에 대한 추천 리스트를 뽑아보기

In [99]:
best_match_vector

array([1, 1, 1, ..., 0, 0, 0])

In [111]:
vector

array([1, 1, 1, ..., 0, 0, 0])

In [112]:
max(vector)

1

In [116]:
recommendation = []
for i, log in enumerate(zip(vector, best_match_vector)):
  log1, log2 = log # 유저 0이 본 영화정보 vector(log1)와 유저 0과 비슷한 유저가 본 영화정보 best_match_vector(log2)를 비교하기 위함
  # 기존에 본 영화(log1)은 배제하고(0인 경우), 보지 않았던 영화 중, 가장 유사한 유저가 본 영화(log2가 1인경우) 
  if log1 < 1. and log2 > 0.:
    recommendation.append(i)
print(recommendation)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

In [93]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset


# model = SVD()

In [94]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)
# raw_data[:,0] -= 1
# raw_data[:,1] -= 1

In [95]:
raw_data

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [96]:
raw_data[:,0] -= 1
raw_data[:,1] -= 1
raw_data

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])